## 6-2 線形回帰（Linear Regression） 

### ライブラリーのインポート

In [ ]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
%matplotlib inline

- math：数学的な計算をするためのライブラリ
- sklearn.linear_model.LinearRegression
  - Sckit Learnに実装される線形回帰モデルでの機械学習の関数
- sklearn.model_selection.train_test_split
  - 訓練データとテストデータを分割する関数
- sklearn.metrics
  - 評価指標を計算する関数群
  - r2_score：決定係数、MSE：平均二乗誤差（Mean Square Error）

### 機械学習データの生成

In [ ]:
np.random.seed(50)
data_x = np.arange(0, 1, 1/float(50)).reshape(-1, 1)  
data_ty = data_x ** 2
data_y = data_ty.reshape(-1, 1) + np.random.rand(len(data_ty), 1) * 0.5
# データの可視化 
plt.scatter(data_x, data_y, marker='o')
plt.show()

- np.random.seed(50)
  - データの再現性を保つため、発生する乱数を固定する。
- data_x に0〜1の50個の値を準備し、基準とする関数（y = x2 ）で計算した data_ty に乱数を加えて、data_y を生成する。
- reshape(-1, 1)
  - 列数を1へ変換する（行数は元のデータから決定）。

### 訓練／テストデータの分割

In [ ]:
train_x, test_xd, train_y, test_yd = train_test_split(
    data_x, data_y, test_size=0.2, random_state=2)
test_d = np.hstack([test_xd, test_yd])
test_x, test_y = np.split(np.sort(test_d, axis=0), 2, axis=1)

- train_test_split(data_x, data_y, test_size=0.2, random_state=2)
  - 訓練データ80%、テストデータ20%に分割する。
- test_x, test_y = np.split(np.sort(test_d, axis=0), 2, axis=1)
  - ランダムに分割されるため、後のグラフ描画用にソートする。

### 機械学習モデルの定義

In [ ]:
# 1次式での回帰モデル作成
TRAIN_X1 = train_x
TEST_X1  = test_x

# 2次式での回帰モデル作成
TRAIN_X2 = np.c_[train_x, train_x**2]
TEST_X2  = np.c_[test_x, test_x**2]

# 4次式での回帰モデル作成
TRAIN_X4 = np.c_[train_x, train_x**2, train_x**3 , train_x**4]
TEST_X4  = np.c_[test_x, test_x**2, test_x**3, test_x**4]

- np.c_[a, b, ･･･]
  - 配列aと配列b ･･･ を結合する。

### 機械学習の実行

In [ ]:
# 1次式モデルで学習実行
model1 = LinearRegression()
run1 = model1.fit(TRAIN_X1, train_y)

# 2次式モデルで学習実行
model2 = LinearRegression()
run2 = model2.fit(TRAIN_X2, train_y)

# 4次式モデルで学習実行
model4 = LinearRegression()
run3 = model4.fit(TRAIN_X4, train_y)

- model = LinearRegression()
  - 線形回帰モデルのインスタンス（model）を作成する。 
- run = model.fit(TRAIN_X, train_y)
  - 訓練データ（TRAIN_X, train_y）を使用して、インスタンス（model）の機械学習を実行する。

### テストデータによる予測結果と誤差の計算

In [ ]:
# テストデータを学習済みモデルに与えて予測結果を取得
pred_y1 = model1.predict(TEST_X1)
pred_y2 = model2.predict(TEST_X2)
pred_y4 = model4.predict(TEST_X4)

# テスト結果と予測結果の誤差を計算
col = ['test', 'pred']
df1 = pd.DataFrame(np.concatenate([test_y, pred_y1], 1), columns=col)
df1['diff'] = df1['pred'] - df1['test']
df2 = pd.DataFrame(np.concatenate([test_y, pred_y2], 1), columns=col)
df2['diff'] = df2['pred'] - df2['test']
df4 = pd.DataFrame(np.concatenate([test_y, pred_y4], 1), columns=col)
df4['diff'] = df4['pred'] - df4['test']

- pred_y = model.predict(TEST_X)
  - テストデータ（TEST_X）を学習済みモデル（model）に与えて予測結果を取得して、pred_yへ代入する。
- df = pd.DataFrame(np.concatenate([test_y, pred_y], 1), columns=･･･)
  - テストデータ（test_y）と予測結果（pred_y）を横に結合してデータフレームへ変換する。
- df1['diff'] = df1['pred'] - df1['test']
  - 予測結果と正解値の差分（diff）を計算する。

### 誤差計算結果と評価指標の出力

In [ ]:
display('1次式モデルの誤差', df1)
display('2次式モデルの誤差', df2)
display('4次式モデルの誤差', df4)

print('R2(1)   =', r2_score(test_y, pred_y1))
print('R2(2)   =', r2_score(test_y, pred_y2))
print('R2(4)   =', r2_score(test_y, pred_y4), '\n')

print('RMSE(1) =', math.sqrt(mean_squared_error(test_y, pred_y1)))
print('RMSE(2) =', math.sqrt(mean_squared_error(test_y, pred_y2)))
print('RMSE(4) =', math.sqrt(mean_squared_error(test_y, pred_y4)))

- r2_score(test_y, pred_y)
  - 決定係数（R2）を求める。
- math.sqrt(mean_squared_error(test_y, pred_y))
  - 平均平方二乗誤差（RMSE）を求める。

### 予測結果のグラフ表示

In [ ]:
# 訓練データとテストデータの散布図
plt.scatter(train_x, train_y, marker='v',label="train")
plt.scatter(test_x, test_y, marker='x',label="test")

# 予測結果のグラフ
plt.plot(test_x, pred_y1, label='model1', color='red')
plt.plot(test_x, pred_y2, label='model2', color='orange')
plt.plot(test_x, pred_y4, label='model4', color='green')

# 描画実行
plt.legend(bbox_to_anchor=(1.05, 1), 
           loc='upper left', borderaxespad=0)
plt.show()

- plt.scatter(train_x, train_y, ･･･)
  - 訓練データ（train_x, trainy）の散布図を描画する。
- plt.scatter(test_x, test_y, ･･･)
  - テストデータ（test_x, test_y）の散布図を描画する。
- plt.plot(test_x, pred_y, ･･･)
  - テストデータと予測結果を折れ線グラフで表示する。